# Image Classification Tasks

In [1]:
from vision.nn.core.residual import ResNet50, ResNet101, ResNet152
import lightning as L
import torch
from torchvision.datasets import MNIST, CIFAR10, CIFAR100
from torchvision import transforms
from torchmetrics import Accuracy
from torch.utils.data import DataLoader, random_split
from lightning.pytorch.loggers import TensorBoardLogger
from lightning.pytorch.callbacks import EarlyStopping, ModelSummary


2023-06-23 17:55:29.718669: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-23 17:55:29.797284: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-06-23 17:55:30.161578: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-11.7/lib64:/opt/ros/humble/opt/rviz_ogre_vendor/lib:/opt/ros/humble/lib/x86_64-lin

In [2]:
class ImageClassificationModel(L.LightningModule):
    DATA_DIR = "./data/"
    TRAIN_RATIO = 0.9
    NUM_CLASSES = 100
    NUM_CHANNELS = 3
    DATASETS = {
        "mnist": {"class": MNIST, "num_classes": 10, "num_channels": 1},
        "cifar10": {"class": CIFAR10, "num_classes": 10, "num_channels": 3},
        "cifar100": {"class": CIFAR100, "num_classes": 100, "num_channels": 3},
    }
    MODELS = {"resnet50": ResNet50, "resnet101": ResNet101, "resnet152": ResNet152}

    def __init__(self, model_name: str, dataset_name: str, batch_size=32):
        super().__init__()
        self.dataset = self.DATASETS[dataset_name]
        self.model = self.MODELS[model_name](
            num_classes=self.dataset["num_classes"],
            in_channels=self.dataset["num_channels"],
        )
        self.dataset_name = dataset_name
        self.batch_size = batch_size

        self.transform = transforms.Compose(
            [
                transforms.ToTensor(),
                transforms.Normalize((0.1307,), (0.3081,)),
            ]
        )

        self.train_accuracy = Accuracy(task="multiclass", num_classes=self.NUM_CLASSES)
        self.val_accuracy = Accuracy(task="multiclass", num_classes=self.NUM_CLASSES)
        self.test_accuracy = Accuracy(task="multiclass", num_classes=self.NUM_CLASSES)

    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = torch.nn.functional.cross_entropy(logits, y)
        preds = torch.argmax(logits, dim=1)
        self.train_accuracy.update(preds, y)

        self.log("train_loss", loss)
        self.log("train_acc", self.train_accuracy)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = torch.nn.functional.cross_entropy(logits, y)
        preds = torch.argmax(logits, dim=1)
        self.val_accuracy.update(preds, y)

        self.log("val_loss", loss, prog_bar=True)
        self.log("val_acc", self.val_accuracy, prog_bar=True)

    def test_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = torch.nn.functional.cross_entropy(logits, y)
        preds = torch.argmax(logits, dim=1)
        self.test_accuracy.update(preds, y)

        self.log("test_loss", loss, prog_bar=True)
        self.log("test_acc", self.test_accuracy, prog_bar=True)

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=1e-3)

    def prepare_data(self):
        # download
        self.dataset["class"](self.DATA_DIR, train=True, download=True)
        self.dataset["class"](self.DATA_DIR, train=False, download=True)

    def setup(self, stage=None):
        # Assign train/val datasets for use in dataloaders
        if stage == "fit" or stage is None:
            data_full = self.dataset["class"](self.DATA_DIR, train=True, transform=self.transform)
            len_train = int(len(data_full) * self.TRAIN_RATIO)
            len_val = len(data_full) - len_train
            self.data_train, self.data_val = random_split(data_full, [len_train, len_val])

        # Assign test dataset for use in dataloader(s)
        if stage == "test" or stage is None:
            self.data_test = self.dataset["class"](
                self.DATA_DIR, train=False, transform=self.transform
            )

    def train_dataloader(self):
        return DataLoader(self.data_train, batch_size=self.batch_size, num_workers=10)

    def val_dataloader(self):
        return DataLoader(self.data_val, batch_size=self.batch_size, num_workers=10)

    def test_dataloader(self):
        return DataLoader(self.data_test, batch_size=self.batch_size)


In [3]:
def run(model_name, dataset_name, epochs=3, batch_size=32):
    model = ImageClassificationModel(
        model_name=model_name, dataset_name=dataset_name, batch_size=batch_size
    )

    trainer = L.Trainer(
        max_epochs=epochs,
        accelerator="auto",
        devices=1,
        logger=TensorBoardLogger(f"./experiments/{dataset_name}", name=model_name),
        callbacks=[EarlyStopping(monitor="val_loss", patience=10), ModelSummary(max_depth=1)],
    )
    trainer.fit(model)
    trainer.test()


In [4]:
model_names = ["resnet50", "resnet101", "resnet152"]
for model_name in model_names:
    run(model_name, "cifar10", epochs=100, batch_size=128)


KeyError: 'model_name'